<img src="images/array.png" width="25%" align="right">

# Arrays

## Blocked Algorithms

A *blocked algorithm* executes on a large dataset by breaking it up into many small blocks.

## Blocked Algorithms

* Example: consider taking the sum of a billion numbers
 * Break up the array into 1,000 chunks
 * Take the sum of each chunk
 * Then take the sum of the intermediate sums

**Create random dataset**

In [ ]:
import dask.array as da
import os
import numpy as np

def random_array():
    if os.path.exists('random.hdf5'):
        return

    import h5py

    with h5py.File('random.hdf5') as f:
        dset = f.create_dataset('/x', shape=(1000000000,), dtype='f4')
        for i in range(0, 1000000000, 1000000):
            dset[i: i + 1000000] = np.random.exponential(size=1000000)

In [ ]:
# create data if it doesn't already exist
random_array()  

# Load data with h5py
# this creates a pointer to the data, but does not actually load
import h5py
import os
f = h5py.File('random.hdf5', mode='r')
dset = f['/x']

### Compute sum using blocked algorithm

In [ ]:
# Compute sum of large array, one million numbers at a time
sums = []
for i in range(0, 1000000000, 1000000):
    chunk = dset[i: i + 1000000]  # pull out numpy array
    sums.append(chunk.sum())

total = sum(sums)
print(total)

## `dask.array` for blocked algorithms

* NumPy-like library for large datasets
* extends full N-Dimensional algorithms
* decent subset of the NumPy interface

## `dask.array` object

Use `da.from_array` to construct a blocked array:

1.  `data`: Any object that supports NumPy slicing
2.  `chunks`: A chunk size to tell us how to block

In [ ]:
import dask.array as da
x = da.from_array(dset, chunks=(1000000,))

### What's the difference?

* `dask_array.sum()` builds an expression of the computation
* `numpy_array.sum()` computes the sum immediately

### Why the difference?

* Not necessarily on the same machine
* Only communicate results if really necessary
* The answer may come from only a slice, then compute only that.

**Compute result**

In [ ]:
result = x.sum()
result

In [ ]:
result.compute()

Performance comparision
---------------------------

The following experiment was performed on a heavy personal laptop.  Your performance may vary.  If you attempt the NumPy version then please ensure that you have more than 4GB of main memory.

## NumPy

Needs gigabytes of memory

In [ ]:
%%time 
x = np.random.normal(10, 0.1, size=(20000, 20000)) 
y = x.mean(axis=0)[::100] 
y

## Dask Array

Needs megabytes of memory

In [ ]:
%%time
x = da.random.normal(10, 0.1, size=(20000, 20000), chunks=(1000, 1000))
y = x.mean(axis=0)[::100] 
y.compute()

### Limitations

Notably dask.array has the following failings:

1.  Not all of ``np.linalg``
2.  No support of operations where the resulting shape
    depends on the values of the array.
3.  Operations like ``sort`` which are difficult to do in parallel
4.  Dask development is driven by immediate need